#CMSC421 Final Project
Team Members: Jordan Maggin, Lex Kim, Zhuo Cheng Xie, Neel Jay, Hylene Wu

Project Description: The program will train on the set of images and gather information on airplanes from these images. With this information, it will attempt to both locate and identify airplanes in different sets of videos.


Installing YOLO

In [1]:
!nvidia-smi

Mon May  6 23:19:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.65                 Driver Version: 546.65       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti   WDDM  | 00000000:01:00.0  On |                  N/A |
| 35%   30C    P0              N/A /  75W |    732MiB /  4096MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.3.0+cpu CPU (Intel Core(TM) i7-6700 3.40GHz)
Setup complete  (8 CPUs, 31.9 GB RAM, 1286.3/1862.4 GB disk)


In [3]:
from ultralytics import YOLO
from IPython.display import display, Image


Importing dataset from Roboflow Project

In [16]:
import os
root_dir = os.getcwd()
!mkdir {root_dir}\datasets
%cd {root_dir}\datasets

# paste the exported dataset code here
!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="nr1RFEWRZXe3o3eQV1ns")
project = rf.workspace("cmsc421-final-project").project("vehicle-detection-ansgq")
version = project.version(1)
model = version.model
dataset = version.download("yolov8")

C:\Users\wizar.GW\Downloads\final421\datasets\datasets\datasets
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Vehicle-Detection-1 in yolov8:: 100%|██████████| 13426/13426 [00:08<00:00, 1511.74it/s]


Custom Training

(Weights are saved to /content/runs/detect/train)

# Backend Functions

Could be included in a Flask API if app is ever deployed.

In [ ]:
!pip install ultralytics==8.0.196 roboflow supervision --quiet

In [ ]:
import cv2
import numpy as np

from ultralytics import YOLO
from roboflow import Roboflow

import supervision as sv

In [ ]:
API_KEY = "nr1RFEWRZXe3o3eQV1ns"
roboflow_workspace = "cmsc421-final-project"
roboflow_project = "plane-detection-eyzak"
version = 1

In [ ]:
def get_roboflow_model(workspace, project, version=1):
    rf = Roboflow(api_key=API_KEY)
    model = rf.workspace(workspace).project(project).version(version).model
    return model
model = get_roboflow_model(roboflow_workspace, roboflow_project, version=1)

In [ ]:
def annotate_image(model, image_path, output_path):

    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()

    image = cv2.imread(image_path)

    pred = model.predict(image_path, confidence=0.5, overlap=0.5).json()
    detections = sv.Detections.from_inference(pred)

    labels = [
        f"{class_name} {confidence:.2f}"
        for class_name, confidence
        in zip(detections['class_name'], detections.confidence)
    ]

    annotated_image = bounding_box_annotator.annotate(
        scene=image, detections=detections
    )
    annotated_image = label_annotator.annotate(
        annotated_image, detections=detections, labels=labels
    )

    cv2.imwrite(output_path, annotated_image)


In [ ]:
def annotate_video(model, video_path, output_path):

    tracker = sv.ByteTrack()
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()
    trace_annotator = sv.TraceAnnotator(trace_length=80, thickness=3)
    heat_map_annotator = sv.HeatMapAnnotator(radius=60, kernel_size=35)

    video_info = sv.VideoInfo.from_video_path(video_path)
    frames_generator = sv.get_video_frames_generator(video_path)

    with sv.VideoSink(output_path, video_info) as sink:
        for i, frame in enumerate(frames_generator):
            print(f"Frame {i}")
            pred = model.predict(frame, confidence=0.5, overlap=0.5).json()

            """
            detections = sv.Detections(
                xyxy=pred['predictions'][0]['x'],
                confidence=pred['predictions'][0]['confidence'],
                class_id=pred['predictions'][0]['class']
            )
            """
            detections = sv.Detections.from_inference(pred)
            detections = tracker.update_with_detections(detections)
            labels = [
                f"{class_name} {confidence:.2f}"
                for class_name, confidence
                in zip(detections['class_name'], detections.confidence)
            ]

            annotated_frame = bounding_box_annotator.annotate(
                scene=frame.copy(), detections=detections
            )
            annotated_frame = label_annotator.annotate(
                scene=annotated_frame, detections=detections, labels=labels
            )
            annotated_frame = trace_annotator.annotate(
                scene=annotated_frame, detections=detections
            )
            annotated_frame = heat_map_annotator.annotate(
                scene=annotated_frame, detections=detections
            )

            sink.write_frame(annotated_frame)

In [ ]:
# Example use
video_path = "test_video.mov"
output_path = "annotated_video.mp4"
annotate_video(model, video_path, output_path)

# Simple GUI with PySimpleGUI

In [8]:
# Installing PySimpleGUI
!python -m pip install pysimplegui
!pip install pillow
!pip install inference
!pip install inference[yolo-world]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

pylint 2.16.2 requires colorama>=0.4.5; sys_platform == "win32", but you have colorama 0.4.4 which is incompatible.


     ---------------------------------------- 0.0/40.7 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.7 kB ? eta -:--:--
     ---------------------------- --------- 30.7/40.7 kB 259.2 kB/s eta 0:00:01
     -------------------------------------- 40.7/40.7 kB 324.0 kB/s eta 0:00:00


   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.4 MB 15.2 MB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 10.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/755.6 kB ? eta -:--:--
   ------------------------------ -------- 593.9/755.6 kB 12.4 MB/s eta 0:00:01
   ---------------------------------------- 755.6/755.6 kB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/54.4 kB ? eta -:--:--
   ---------------------------------------- 54.4/54.4 kB ? eta 0:00:00
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.5
    Uninstalling wcwidth-0.2.5:
      Successfully uninstalled wcwidth-0.2.5
  Attempting uninstal

In [13]:
# Imports
import PySimpleGUI as sg
import cv2
import numpy as np
from PIL import Image, ImageTk
import io
import os
from inference import get_model

model_inf = get_model(model_id="plane-detection-eyzak/2", api_key="nr1RFEWRZXe3o3eQV1ns")

In [20]:
# Defines potential filetypes to search for
file_types = [("JPEG (*.jpg)", "*.jpg"),
              ("All files (*.*)", "*.*")]

def main():
    # Defines list of files from selected folder
    file_list_column = [
        [
            sg.Text("Image Folder"),
            sg.In(size=(25, 1), enable_events=True, key="-FOLDER-"),
            sg.FolderBrowse(),
        ],
        [
            sg.Listbox(
                values=[], enable_events=True, size=(40, 20), key="-FILE LIST-"
            )
        ],
    ]
    
    # File loader // UNUSED
    file_loader_column = [
        [
            sg.Text("Image File"),
            sg.Input(size=(25, 1), key="-FILE-"),
            sg.FileBrowse(file_types=file_types),
        ],
        [
            sg.Button("Load Image"),
        ]
    ]
    
    # Image display
    image_display_column = [
        [
            sg.Image(key="-IMAGE-"),
        ]
    ]
    
    # Defines the layout of the program
    layout = [
        #[sg.Image(key="-IMAGE-")],
        [
            sg.Column(file_list_column, element_justification='c'),
#             sg.Column(file_loader_column, element_justification='c'),
            sg.VSeparator(),
            sg.Column(image_display_column, element_justification='c'),
        ],
#         [
#             sg.Button("Confusion Matrix"),
#             sg.Button("Result Graphs"),
#             sg.Button("Predictions"),
#         ],
    ]
    
    
    # Applies the layout to the window
    window = sg.Window("Image Viewer", layout, element_justification='c')
    
    ###########
    # UPDATER #
    ###########
    # Updates the window with corresponding values and images
    while True:
        event, values = window.read()
        if event == "Exit" or event == sg.WIN_CLOSED:
            break
        if event == "-FOLDER-":
            folder = values["-FOLDER-"]
            try:
                # Get list of files in folder
                file_list = os.listdir(folder)
            except:
                file_list = []

            fnames = [
                f
                for f in file_list
                if os.path.isfile(os.path.join(folder, f))
                and f.lower().endswith((".jpg"))
            ]
            window["-FILE LIST-"].update(fnames)
        if event == "-FILE LIST-":  # A file was chosen from the listbox
            try:
                filename = os.path.join(
                    values["-FOLDER-"], values["-FILE LIST-"][0]
                )
                image = Image.open(filename)
                image.thumbnail((400, 400))
                bio = io.BytesIO()
                image.save(bio, format="PNG")
                window["-IMAGE-"].update(data=bio.getvalue())
            except:
                pass
        if event == "Load Image":  # NOTE: This part is currently unused, but is available for debugging.
            filename = values["-FILE-"]
            if os.path.exists(filename):
                # NOTE: If you want to do predictions, they would have to be placed here, prior to conversion.
                image = Image.open(values["-FILE-"])
                image.thumbnail((400, 400))
                bio = io.BytesIO()
                image.save(bio, format="PNG")
                window["-IMAGE-"].update(data=bio.getvalue())
        if event == "Confusion Matrix":
            window["-IMAGE-"].update(confusion)
        if event == "Result Graphs":
            window["-IMAGE-"].update(results)
        if event == "Predictions":
            window["-IMAGE-"].update(predictions)
    
    # Closes window as necessary
    window.close()

main()